In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# 모델은 Q개의 label을 출력
# 논문 상에서 activate function은 tanh
class Model(nn.Module):
    
    def __init__(self, num_features, hid_dim, Q):
        super().__init__()
        self.layer1 = nn.Linear(num_features, hid_dim, )
        self.layer2 = nn.Linear(hid_dim, Q)
        
    def forward(self, x):
        out = torch.tanh(self.layer1(x))
        return torch.tanh(self.layer2(out))

In [3]:
class BPMLLLoss(nn.Module):

    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super().__init__()
        if size_average is not None or reduce is not None:
            self.reduction = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction

    def forward(self, input, target):
        # https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/tkde06a.pdf
        # input's shape is (bsz, Q)
        # target's shape is (bsz, Q)
        # target's unique value is in [-1, 1]
    
        loss_sum = 0
        for i in range(len(target)) : 
            Y = input[i][target[i] == 1]
            Y_bar = input[i][target[i] != 1]
            loss = sum(torch.exp(-torch.cartesian_prod(Y, -Y_bar).sum(dim=-1)))/ len(torch.cartesian_prod(Y, Y_bar))
            loss_sum += loss
        return loss_sum
        

In [5]:
BPMLL_network = Model(784,100,5)

In [8]:
# loss, optimizer
criterion = BPMLLLoss() 
optimizer = optim.SGD(BPMLL_network.parameters(), lr=0.01)